In [ ]:
import os
from dotenv import load_dotenv

In [ ]:
load_dotenv(override=True, dotenv_path="../.env.local")
my_api_key = os.getenv("OPENAI_API_KEY")

print (my_api_key)

In [ ]:
from openai import OpenAI   
# client = OpenAI(api_key=my_api_key)
client = OpenAI()  # API key will be read from environment variable OPENAI_API_KEY

In [ ]:
def ask_question(prompt: str) -> str:
    """
    Ask a question to GPT, with moderation check.
    If the prompt is flagged, it will not be sent to the chat model.
    """
    print(f"User asked: {prompt}")

    # Step 1: Run moderation check
    mod_result = client.moderations.create(
        model="omni-moderation-latest",
        input=prompt
    )

    print(f"Moderation result: {mod_result}")
    print(f"mod_result.results[0].flagged result: {mod_result.results[0].flagged}")
    # Check if flagged
    if mod_result.results[0].flagged:
        print("Prompt was flagged by moderation. Blocking response.")
        return "Sorry, your request cannot be processed as it violates safety policies."
    else:
        print("Prompt passed moderation.")
        # Step 2: Only query GPT if safe
        response = client.chat.completions.create(
            model="gpt-5-nano",  # Use your desired model
            messages=[
                {"role": "system", "content": "You are a helpful assistant. Answer as concisely as possible."},
                {"role": "user", "content": prompt}
            ]
        )

        answer = response.choices[0].message.content
        print(f"Assistant replied: {answer}")
        return answer

In [ ]:
# print(ask_question("How do I bake a chocolate cake?"))
print(ask_question("You're really annoying and I don’t want to work with you anymore."))  # Will be flagged by moderation